In [ ]:
# Sequencing

# This model uses the basal ganglia model to cycle through a sequence of five 
# states (i.e., A->B->C->D->E->A->...). The model incorporates a 
# working memory component (state), which allows the basal ganglia to update 
# that memory based on a set of condition/action mappings.

# The model has parameters as described in the book. In Nengo 1.4 (and the book) 
# separate 'Rules' and 'Sequence' classes were created. However, this is not 
# needed in Nengo 2.0 since you can directly specify the rules using the 
# built-in 'Actions' class in the spa (semantic pointer architecutre) package. This 
# class takes a string definition of the action as an input as shown in the code 
# where '-->' is used to split the action into condition and effect, otherwise 
# it is treated as having no condition and just effect.

# The syntax for creating an input function in Nengo 2.0 is also different from 
# that in Nengo 1.4 mentioned in the book. The syntax for Nengo 2.0 which is 
# used here is spa.input(module=function). The first parameter 'module' 
# refers to name of the module that you want to provide input to and the second 
# parameter 'function' refers to the function to execute to generate inupts to 
# that module. The functions should always return strings, which will then be 
# parsed by the relevant vocabulary.
#Setup the environment
import nengo                
import nengo_spa as spa       #import spa related packages

#Number of dimensions for the Semantic Pointers
dimensions = 16

#Make a model with the SPA network
with spa.Network(label="Sequence") as model:
    
    #Creating a working memory/cortical element 
    state = spa.State(vocab=dimensions, feedback=1, feedback_synapse=0.01, label="WM")

    #Function that provides the model with an initial input semantic pointer.
    def start(t):
        if t < 0.1:          #Duration of the initial input = 0.1
            return 'D'
        else:
            return '0'
        
    #Input
    visual = spa.Transcode(start, label="visual", output_vocab=dimensions)
    visual >> state
    
    # Actions
    with spa.ActionSelection():
        spa.ifmax(spa.dot(state, spa.sym.A), spa.sym.B >> state)
        spa.ifmax(spa.dot(state, spa.sym.B), spa.sym.C >> state)
        spa.ifmax(spa.dot(state, spa.sym.C), spa.sym.D >> state)
        spa.ifmax(spa.dot(state, spa.sym.D), spa.sym.E >> state)
        spa.ifmax(spa.dot(state, spa.sym.E), spa.sym.A >> state)